<a href="https://colab.research.google.com/github/Elhai12/SBA_loans_predict/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title don't show code till dblclick
#Clone git to get the files
!git clone https://github.com/Elhai12/SBA_loans_predict.git

Cloning into 'SBA_loans_predict'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 263 (delta 89), reused 24 (delta 24), pack-reused 113 (from 1)
Receiving objects: 100% (263/263), 39.22 MiB | 19.03 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [2]:
#@title don't show code till dblclick
import pandas as pd
import numpy as np
from joblib import dump,load
import ipywidgets as widgets
from IPython.display import display

load_model = load('/content/SBA_loans_predict/model_sba_predict.joblib')
load_label = load('/content/SBA_loans_predict/label_encoders.joblib')
load_onehot = load('/content/SBA_loans_predict/one_hot_encoder.joblib')

In [3]:
#@title don't show code till dblclick
#Define numeric widgets
style = {'description_width': 'initial'}
widgets_dict_num={}
rows_num = []
columns_mapping = {
    'ApprovalFY': 'Fiscal Year',
    'Term': 'Loan Term',
    'NoEmp': 'Employee Count',
    'CreateJob': 'Jobs Created',
    'RetainedJob': 'Jobs Retained',
    'DisbursementGross': 'Disbursed Amount',
    'GrAppv': 'Loan Approved',
    'SBA_Appv': 'SBA Approved',
    'Month_apv': 'Approval Month',
    'Day_apv': 'Approval Day',
    'Diff_days': 'Days Difference',
    'FEDFUNDS': 'Federal Funds',
    'change_rate_fed': 'Rate Change'

}
for col,desc in columns_mapping.items():
  widget = widgets.FloatText(description=desc,style=style)
  widgets_dict_num[col] = widget
  rows_num.append(widget)

layout_num = widgets.VBox(rows_num)


columns_options = {
    "UrbanRural": ['Undefined', 'Urban', 'Rural'],
    "RevLineCr": ['N', 'Y'],
    "LowDoc": ['N', 'Y'],
    "Bank_State_group": ['FL', 'other', 'SD', 'NY', 'CA', 'NC', 'GA', 'IL', 'TX', 'VA'],
    "Region": ['East Florida', 'South Florida', 'Central Florida', 'North Florida', 'West Florida'],
    "Category_bank": ['Other', 'Commercial Bank', 'Capital/Finance Institution', 'Development Institution', 'Credit Union', 'International'],
    "Sector_group": ['Information and Communication', 'Culture, Leisure, and Restaurants', 'Public Services and Health', 'Construction and Infrastructure', 'Professional and Scientific Services', 'Natural Resources and Energy', 'Finance and Insurance', 'Manufacturing', 'Transportation and Logistics', 'Agriculture, forestry, fishing and hunting']
}


widgets_dict_cat = {}
rows_cat = []

for col, options in columns_options.items():
    widget = widgets.Dropdown(
        options=options,
        description=col,
        style=style
    )
    widgets_dict_cat[col] = widget
    rows_cat.append(widget)


layout_cat = widgets.VBox(rows_cat)

In [4]:
#@title don't show code till dblclick
for row_one in range(0,len(rows_num),3):
  display(widgets.HBox(rows_num[row_one:row_one+3]))


In [5]:
#@title don't show code till dblclick
for row_one in range(0,len(rows_cat),3):
  display(widgets.HBox(rows_cat[row_one:row_one+3]))

In [17]:
#@title don't show code till dblclick
dict_df_cat = {}
dict_df_num = {}
for col,widget in widgets_dict_cat.items():
  dict_df_cat[col] = widget.value
for col,widget in widgets_dict_num.items():
  dict_df_num[col] = widget.value

dict_all = {**dict_df_num,**dict_df_cat}
df = pd.DataFrame(dict_all,index=[0])
df['UrbanRural'] = df['UrbanRural'].map({'Undefined':0,'Urban':1,'Rural':2}).astype(int)


cat_cols =  dict_df_cat.keys()
cols_labels = ['Sector_group','Category_bank','Region','Bank_State_group']
cols_onehot = [col for col in cat_cols if col not in cols_labels]
for col in cols_labels:
  df[col] = load_label[col].transform([df[col]])

one_hot_encoded = load_onehot.transform(df[cols_onehot],)
one_hot_df = pd.DataFrame(one_hot_encoded, columns=load_onehot.get_feature_names_out(cols_onehot))

df_all = pd.concat([df,one_hot_df],axis=1)
df_final = df_all.drop(columns=cols_onehot)

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was

In [18]:
#@title don't show code till dblclick
Probalbilty = load_model.predict_proba(df_final)[:,-1]
print(f'{round(Probalbilty.tolist()[0]*100,3)}%')

22.72%
